In [1]:
import os
os.getcwd()

'C:\\Users\\clari\\Desktop\\M2 - Thesis\\Research\\Dr Jacques Bou Abdo\\Recommender System\\5 - Ensemble Learning Model\\Accuracy Metrics\\serendipity-main\\models\\lightgcn'

In [2]:
#dataset_name = 'framework-ml-25m-subset'
#               ml-25m-subset(3)-#5
dataset_name = 'framework-ml-25m-subset(3) (EL2.4)'
#dataset_name = 'ml-25m-subset(3)-#5'
#dataset_name = 'ml-25m-subset(3)-#2.3'
RATINGS_PATH = '../../datasets/'+dataset_name+'/clean/ratings.csv'
otp = 'output_CH/'
#RATINGS_PATH = inp + dataset_name + "\\clean\\ratings.csv"

In [3]:
import pandas as pd
import torch
import numpy
from torch.utils.data import DataLoader, Dataset
from torch import nn

# Define a simple LightGCN class
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=64, num_layers=3):
        super(LightGCN, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # Initialize user and item embeddings
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)
        nn.init.xavier_uniform_(self.user_embeddings.weight)
        nn.init.xavier_uniform_(self.item_embeddings.weight)

    def forward(self, user_indices, item_indices):
        # Initial embeddings
        user_emb = self.user_embeddings(user_indices)
        item_emb = self.item_embeddings(item_indices)
        return user_emb, item_emb

In [4]:
import numpy
import pandas as pd

# Load your ratings data
ratings = pd.read_csv(RATINGS_PATH).rename(columns={'movieId':'itemId'})
print(len(ratings))
if '#2' in dataset_name:
    ratings = ratings[(ratings['isNoisy'] == "[0.]") | (ratings['isNoisy'] == "0")]
elif 'nf-1' in dataset_name:
    #ratings =  ratings[(ratings['1&2&3&4 = 0'] == 0) & (ratings['1&2&3&4 = 1'] == 0)]
    ratings =  ratings[(ratings['1&2&3&4 = 1'] == 0)]
#elif ('nf1' in dataset_name) or ('nf2' in dataset_name) or ('nf3' in dataset_name) or ('nf4' in dataset_name):
#    ratings = ratings
elif (not 'framework' in dataset_name) & ('ml-25m-subset(3)-' in dataset_name):
    ratings = ratings[ratings['isNoisy'] == 0]
elif 'framework' in dataset_name:
    ratings = ratings[ratings['layer3_result'] == 0]



634341


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107084 entries, 15 to 634319
Data columns (total 38 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0.2   107084 non-null  int64  
 1   Unnamed: 0.1   107084 non-null  int64  
 2   userId         107084 non-null  int64  
 3   itemId         107084 non-null  int64  
 4   rating         107084 non-null  float64
 5   timestamp      107084 non-null  int64  
 6   user_cat       107084 non-null  object 
 7   rating_group   107084 non-null  object 
 8   item_cat       107084 non-null  object 
 9   nf1            107084 non-null  int64  
 10  user_group     107084 non-null  object 
 11  coherence      107084 non-null  float64
 12  title          107084 non-null  object 
 13  genres         107084 non-null  object 
 14  RND            107084 non-null  float64
 15  thresh         107084 non-null  float64
 16  nf2            107084 non-null  int64  
 17  Unnamed: 0     107084 non-null  i

In [6]:
print(len(ratings))

107084


In [7]:
# Create mappings for users and items
user_mapping = {id: idx for idx, id in enumerate(ratings['userId'].unique())}
item_mapping = {id: idx for idx, id in enumerate(ratings['itemId'].unique())}

ratings['user_idx'] = ratings['userId'].map(user_mapping)
ratings['item_idx'] = ratings['itemId'].map(item_mapping)

num_users = len(user_mapping)
num_items = len(item_mapping)

# Create a dataset class for training
class RatingDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        user = self.ratings.iloc[idx]['user_idx']
        item = self.ratings.iloc[idx]['item_idx']
        return torch.tensor(user, dtype=torch.long), torch.tensor(item, dtype=torch.long)

dataset = RatingDataset(ratings)
data_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

# Initialize the LightGCN model
model = LightGCN(num_users, num_items)

# Training loop (simplified version)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(10):  # Example with 10 epochs
    for user_indices, item_indices in data_loader:
        optimizer.zero_grad()
        user_emb, item_emb = model(user_indices, item_indices)
        loss = (user_emb - item_emb).pow(2).mean()
        loss.backward()
        optimizer.step()

# Save embeddings to CSV
#user_embeddings = model.user_embeddings.weight.detach().cpu().numpy()
#item_embeddings = model.item_embeddings.weight.detach().cpu().numpy()

user_embeddings = model.user_embeddings.weight.detach().numpy()
item_embeddings = model.item_embeddings.weight.detach().numpy()

#torch.save(model.user_embeddings.weight.detach().cpu(), 'user_embeddings.pt')
#torch.save(model.item_embeddings.weight.detach().cpu(), 'item_embeddings.pt')

user_df = pd.DataFrame(user_embeddings, index=user_mapping.keys())
item_df = pd.DataFrame(item_embeddings, index=item_mapping.keys())



'''
try:
    user_embeddings = model.user_embeddings.weight.detach().cpu().numpy()
    item_embeddings = model.item_embeddings.weight.detach().cpu().numpy()
    user_df = pd.DataFrame(user_embeddings, index=user_mapping.keys())
    item_df = pd.DataFrame(item_embeddings, index=item_mapping.keys())
    user_df.to_csv('user_embedding_CH.csv')
    item_df.to_csv('item_embedding_CH.csv')
except ImportError:
    print("NumPy or Pandas not available. Saved as PyTorch tensors.")
'''


'\ntry:\n    user_embeddings = model.user_embeddings.weight.detach().cpu().numpy()\n    item_embeddings = model.item_embeddings.weight.detach().cpu().numpy()\n    user_df = pd.DataFrame(user_embeddings, index=user_mapping.keys())\n    item_df = pd.DataFrame(item_embeddings, index=item_mapping.keys())\n    user_df.to_csv(\'user_embedding_CH.csv\')\n    item_df.to_csv(\'item_embedding_CH.csv\')\nexcept ImportError:\n    print("NumPy or Pandas not available. Saved as PyTorch tensors.")\n'

In [8]:
item_df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
30812,-0.003382,-0.007927,-0.012791,0.006440,-0.006063,-0.002752,0.007867,0.003004,-0.001990,-0.000872,...,-0.004355,0.003541,0.000842,0.000336,0.000727,0.002702,-0.000550,-0.009633,-0.000432,-0.003852
2353,-0.005206,-0.019404,-0.010282,-0.006031,0.001181,0.016274,0.012071,0.000728,-0.019885,0.022068,...,0.007383,-0.004535,-0.015098,0.003748,-0.006119,-0.004245,0.023018,-0.001427,-0.003798,0.011743
1645,0.031058,-0.005008,0.035542,0.018180,0.018018,0.024922,0.016597,0.001992,-0.020592,0.006128,...,-0.028215,0.016821,0.001081,0.013601,0.016344,-0.000074,0.052783,-0.014248,-0.008143,-0.016399
36517,0.004572,-0.018915,-0.017203,0.021992,-0.028250,0.000657,0.008045,0.001630,-0.013293,-0.030316,...,-0.001789,-0.035111,0.018306,-0.005790,0.023170,-0.007304,0.028234,-0.028824,-0.003474,0.022613
2181,0.015247,-0.012352,0.048768,-0.038547,0.047754,-0.035440,-0.009466,-0.012421,0.003336,0.017260,...,-0.043801,-0.018412,-0.029922,-0.009168,0.013003,-0.021407,0.001182,-0.073660,0.015685,0.042281


In [9]:
user_df.head()
print(len(user_df))

5522


In [10]:
user_df.to_csv(otp + dataset_name + '/user_embedding_CH.csv')
item_df.to_csv(otp + dataset_name +'/item_embedding_CH.csv')

In [11]:
print(len(item_df))

10339
